In [4]:
# Cell 1: scripts folder to path & import modules for whole coding
import sys
from pathlib import Path
import time
from datetime import timedelta

# 1) Locate project root (one level up from this notebooks directory)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    project_root = Path.cwd().parent
    
print(f"Project root identified as: {project_root}")

# 2) Point at your scripts folder - FIXED PATH HERE
scripts_dir = project_root / "5_nsga" / "5_nsga_scripts"
assert scripts_dir.exists(), f"Can't find scripts at {scripts_dir}"
sys.path.insert(0, str(scripts_dir))

# 3) Import helper modules
from pv_simulate_30 import simulate_multi_year_pv
from battery_30 import simulate_battery_dispatch
from financial_30 import compute_financials
from objective_30 import evaluate_solution
import objective_30  # Import as module to set global variables

# 4) Import optimiser & data libs
import numpy as np
import pandas as pd
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.core.callback import Callback

print("All modules imported successfully - can move onto next stage")

Project root identified as: /Users/petertunali/Documents/GitHub/Battery_Optimisation
All modules imported successfully - can move onto next stage


In [5]:
# ─── Cell 2: Configure outputs, check data & build 30‑yr demand ────────────────
from pathlib import Path
import pandas as pd

# 1) Auto‑version your outputs folder under project_root/outputs_optimisation
base_out = project_root / "outputs_optimisation"
base_out.mkdir(exist_ok=True)
existing = [d.name for d in base_out.iterdir() if d.is_dir() and d.name.isdigit()]
nums     = sorted(int(n) for n in existing) if existing else []
next_run = nums[-1] + 1 if nums else 1
out_dir  = base_out / f"{next_run:03d}"
out_dir.mkdir()
print(f"Writing all outputs to → {out_dir}")

# 2) Locate your data directory
data_dir = project_root / "data"
if not data_dir.exists():
    for alt in [project_root/"Battery_Optimisation"/"data", Path.cwd()/"data"]:
        if alt.exists():
            data_dir = alt
            break
print(f"Using data directory → {data_dir}")

# 3) Check EPW weather files
weather_files = [
    str(data_dir / "Bonfire_2025.epw"),
    str(data_dir / "Bonfire_2040_4_5.epw"),
    str(data_dir / "Bonfire_2050_4_5.epw")
]
print("\nChecking weather files:")
for wf in weather_files:
    print(" ✔ Found:" if Path(wf).exists() else " ❌ Missing:", wf)
assert all(Path(wf).exists() for wf in weather_files), "One or more weather files missing!"

# 4) Find demand CSV (take the first match)
demand_paths = [
    data_dir / "PV_Generation_excel.csv",
    data_dir / "Energy_Demand_and_Supply_2024.csv"
]
for p in demand_paths:
    if p.exists():
        demand_file = p
        print("\n✔ Found demand file:", p)
        break
else:
    raise FileNotFoundError("No demand file found – check your filenames")

# 5) Load the one‑year demand series
#    - timestamps in col 0, of form '31/12/2024  12:00:00 pm'
#    - exactly one numeric column of consumption
raw = pd.read_csv(demand_file, header=0, dtype=str)

# 5a) Clean & parse the timestamps
ts = raw.iloc[:,0].str.strip()
# strip any duplicated 'am'/'pm'
ts = ts.str.replace(r'.*(am|pm).*$', r'\1', regex=True).str.replace(r'\s+(am|pm)$', r' \1', regex=True)
one_idx = pd.to_datetime(ts, dayfirst=True)

# 5b) Identify the demand column
#    drop col 0 and any fully empty columns, then take the one remaining numeric column
cands = raw.columns[1:]
# coerce to numeric to test
valid = [c for c in cands 
         if pd.to_numeric(raw[c].str.replace(',',''), errors='coerce').notna().any()]
if not valid:
    raise ValueError("Couldn't find any numeric demand column in CSV")
demand_col = valid[0]
one_year = pd.to_numeric(raw[demand_col].str.replace(',',''), errors='coerce')
one_year.index = one_idx

# 5c) Sanity‑check: should be 17 520 half‑hour points
if len(one_year) != 17520:
    print(f"⚠️  Loaded {len(one_year)} points (expected 17520). Check your source file.")

print(f"\nUsing demand column → '{demand_col}'")
print(" First few entries:")
print(one_year.head())

# 6) Tile that one‑year series into a 30‑yr profile (3 EPWs × 10 repeats each)
REPEATS_PER_FILE = 10
n_repeats        = REPEATS_PER_FILE * len(weather_files)
tiled = pd.concat([one_year.copy() for _ in range(n_repeats)], ignore_index=True)

# 7) Build a continuous half‑hourly datetime index from the original start
start = one_year.index[0]
full_idx = pd.date_range(start, periods=len(tiled), freq="30min")
tiled.index = full_idx
demand_profile = tiled

# 8) Summary
print(f"\n30‑year demand profile built:")
print(f"  • Time steps : {len(demand_profile)}")
print(f"  • Date range : {demand_profile.index[0]} → {demand_profile.index[-1]}")
print(f"  • Total demand: {demand_profile.sum():.2f} kWh")


Writing all outputs to → /Users/petertunali/Documents/GitHub/Battery_Optimisation/outputs_optimisation/001
Using data directory → /Users/petertunali/Documents/GitHub/Battery_Optimisation/data

Checking weather files:
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2025.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2040_4_5.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2050_4_5.epw

✔ Found demand file: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv
⚠️  Loaded 17570 points (expected 17520). Check your source file.

Using demand column → 'PV Generated (kWh)'
 First few entries:
Date and Time
2025-01-01 00:00:00    0.0
2025-01-01 00:30:00    0.0
2025-01-01 01:00:00    0.0
2025-01-01 01:30:00    0.0
2025-01-01 02:00:00    0.0
Name: PV Generated (kWh), dtype: float64

30‑year demand profile built:
  • Time steps : 527100
  • Date range : 2025-01-

In [6]:
# Cell 3: Define the PV systems and roof parameters
# Define PV system parameters
roof_params = [{
    'name': 'main_roof',
    'system_capacity_kw': 10.0,  # 10kW system
    'tilt': 10.0,
    'azimuth': 18.0,  # North-facing in Southern Hemisphere
    'shading': 43.0,  # 43% shading impact
}]

print("Roof parameters:")
for roof in roof_params:
    print(f"  - {roof['name']}: {roof['system_capacity_kw']} kW, Tilt: {roof['tilt']}°, Azimuth: {roof['azimuth']}°, Shading: {roof.get('shading', 0.0)}%")

total_pv_capacity = sum(roof['system_capacity_kw'] for roof in roof_params)
print(f"Total PV capacity: {total_pv_capacity} kW")
print("ready for next stage")

Roof parameters:
  - main_roof: 10.0 kW, Tilt: 10.0°, Azimuth: 18.0°, Shading: 43.0%
Total PV capacity: 10.0 kW
ready for next stage


In [7]:
# ─── Cell 4: Build 30‑yr PV & demand, then compute zero‑battery baseline ─────
print("1) Simulating 30 yr PV…")
start_years = [2025, 2040, 2050]
pv_profile = simulate_multi_year_pv(
    weather_files    = weather_files,
    roof_params      = roof_params,
    repeats_per_file = 10,
    start_years      = start_years
)
print(f"   • PV steps : {len(pv_profile)}")
print(f"   • Date      : {pv_profile.index[0]} → {pv_profile.index[-1]}")
print(f"   • Gen total : {pv_profile['simulated_kwh'].sum():.2f} kWh\n")

print("2) Building 30 yr demand by tiling one‑year…")
# (this assumes you’ve loaded “one_year” as your single‑year pd.Series already)
n_repeats     = len(start_years) * 10
demand_tiled  = pd.concat([one_year.copy() for _ in range(n_repeats)], ignore_index=True)
demand_index  = pd.date_range(one_year.index[0], periods=len(demand_tiled), freq='30min')
demand_tiled.index = demand_index
demand_profile   = demand_tiled
print(f"   • Demand steps : {len(demand_profile)}")
print(f"   • Date         : {demand_profile.index[0]} → {demand_profile.index[-1]}")
print(f"   • Demand total : {demand_profile.sum():.2f} kWh\n")

print("3) Zero‑battery baseline dispatch…")
disp0, totals0 = simulate_battery_dispatch(
    pv_gen      = pv_profile['simulated_kwh'],
    demand      = demand_profile,
    battery_kwh = 0.0
)

fin0 = compute_financials(
    totals0,
    battery_kwh          = 0.0,
    pv_kw                = total_pv_capacity,
    discount_rate        = 0.05,
    baseline_import_cost = 0.0     # nothing to subtract yet
)

BASE_IMPORT_COST = fin0['import_cost_total']
print(f"   → Baseline import cost over 30 yr = ${BASE_IMPORT_COST:,.2f}")

# Tell your objective about it:
import objective_30
objective_30.BASE_IMPORT_COST = BASE_IMPORT_COST


1) Simulating 30 yr PV…
   • PV steps : 525600
   • Date      : 2025-01-01 00:00:00 → 2054-12-24 23:30:00
   • Gen total : 204028.14 kWh

2) Building 30 yr demand by tiling one‑year…
   • Demand steps : 527100
   • Date         : 2025-01-01 00:00:00 → 2055-01-25 05:30:00
   • Demand total : 199896.98 kWh

3) Zero‑battery baseline dispatch…
   → Baseline import cost over 30 yr = $27,608.57


In [8]:
# ─── Cell 5: Quick battery dispatch test ──────────────────────────────────────
print("Testing battery simulation with 10 kWh…")
test_battery_kwh = 10.0

disp_df, totals = simulate_battery_dispatch(
    pv_gen      = pv_profile['simulated_kwh'],
    demand      = demand_profile,
    battery_kwh = test_battery_kwh
)

print(f"  • Peak imports  : {totals['total_grid_import_peak']:.1f} kWh")
print(f"  • Off‑peak imps : {totals['total_grid_import_offpeak']:.1f} kWh")
print(f"  • PV exports    : {totals['total_pv_export']:.1f} kWh")


Testing battery simulation with 10 kWh…
  • Peak imports  : 139.5 kWh
  • Off‑peak imps : 202.7 kWh
  • PV exports    : 336.5 kWh


In [9]:
# ─── Cell 6: Compute “no‑battery” baseline import cost ────────────────────────
disp0, totals0 = simulate_battery_dispatch(
    pv_gen      = pv_profile['simulated_kwh'],
    demand      = demand_profile,
    battery_kwh = 0.0
)

fin0 = compute_financials(
    totals0,
    battery_kwh          = 0.0,
    pv_kw                = total_pv_capacity,
    discount_rate        = 0.05,
    baseline_import_cost = 0.0   # zero storage → no subtraction here
)

BASE_IMPORT_COST = fin0['import_cost_total']
print(f"Baseline import cost (no battery over 30 yr) = ${BASE_IMPORT_COST:,.2f}")

# tell objective to use it
import objective_30
objective_30.BASE_IMPORT_COST = BASE_IMPORT_COST


Baseline import cost (no battery over 30 yr) = $27,608.57


In [10]:
# Cell 7: Do financial calculations
# Scale totals to 30 years (multiply by 10 as per your approach)
print("Testing financial calculation")

try:
    # Scale the 3-year totals to 30 years
    totals_30yr = {k: v * 10 for k, v in totals.items()}
    
    # Calculate financials
    fin_results = compute_financials(
        totals=totals_30yr,
        battery_kwh=test_battery_kwh,
        pv_kw=total_pv_capacity,
        discount_rate=0.05  # 5% discount rate
    )
    
    # Display results
    print(f"Financial calculation complete")
    print(f"  - CAPEX (PV): ${fin_results['capex_pv']:,.2f}")
    print(f"  - CAPEX (Battery): ${fin_results['capex_battery']:,.2f}")
    print(f"  - Total CAPEX: ${fin_results['capex_total']:,.2f}")
    print(f"  - Export revenue: ${fin_results['export_revenue_total']:,.2f}")
    print(f"  - Import cost: ${fin_results['import_cost_total']:,.2f}")
    print(f"  - OPEX: ${fin_results['opex_total']:,.2f}")
    print(f"  - Net cost: ${fin_results['net_cost']:,.2f}")
    print(f"  - IRR: {fin_results['irr']*100:.2f}%")
    if fin_results['npv'] is not None:
        print(f"  - NPV: ${fin_results['npv']:,.2f}")
        print("PETER: CHECK AGAIN IRR if NEGATIVE, wont work, remember to redo - and change NPV to NPC")
    
except Exception as e:
    print(f"Error in financial calculation - check again (or redo calcs): {e}")
   

Testing financial calculation
Financial calculation complete
  - CAPEX (PV): $15,000.00
  - CAPEX (Battery): $11,132.10
  - Total CAPEX: $26,132.10
  - Export revenue: $111.04
  - Import cost: $828.06
  - OPEX: $20,805.00
  - Net cost: $47,654.12
  - IRR: nan%
  - NPV: $-37,160.31
PETER: CHECK AGAIN IRR if NEGATIVE, wont work, remember to redo - and change NPV to NPC


In [11]:
# Cell 8: Test the objective function
print("Testing objective function...")

try:
    # Create test parameters
    test_params = {
        'battery_kwh': test_battery_kwh,
        'pv_kw': total_pv_capacity
    }
    
    # Evaluate the objective function
    objective_results = evaluate_solution(
        params=test_params,
        pv_profile=pv_profile,
        demand_profile=demand_profile
    )
    
    # Display results
    print(f"Objective function evaluation successful")
    print(f"  - Negative IRR: {objective_results[0]}")
    print(f"  - NPC: ${objective_results[1]:,.2f}")
    print(f"  - Actual IRR: {-objective_results[0]*100:.2f}%")
    
except Exception as e:
    print(f"Error in objective function: {e}")
    raise

Testing objective function...
Objective function evaluation successful
  - Negative IRR: nan
  - NPC: $17,773.80
  - Actual IRR: nan%


In [12]:
# Cell 9: Define the optimisation problem to solve
print("PETER: ADD SOLAR PV TO VARY FOR EACH ROOF")
print("PETER: TO ADD EACH SIMULATION TO SAY TIME TAKEN")
print("Defining optimization problem")

try:
    # Define the NSGA-II optimization problem (Remember peter to write optimisation with s as python uses american english)
    class BatteryOptimizationProblem(Problem):
        def __init__(self):
            # Define one variable (battery size in kWh)
            # with lower bound 0 and upper bound 50
            super().__init__(
                n_var=1,             # Number of decision variables
                n_obj=2,             # Number of objectives
                n_ieq_constr=0,      # Number of inequality constraints
                xl=np.array([0.0]),  # Lower bounds of variables
                xu=np.array([50.0])  # Upper bounds of variables
            )
            # Store the profiles for use in evaluation
            self.pv_profile = pv_profile
            self.demand_profile = demand_profile
        
        def _evaluate(self, x, out, *args, **kwargs):
            # Evaluate each solution in the population
            n_solutions = x.shape[0]
            F = np.zeros((n_solutions, 2))  # For storing [-IRR, NPC] values
            
            # Loop through each solution (battery size)
            for i in range(n_solutions):
                battery_kwh = x[i, 0]
                # Define parameters for evaluation
                params = {
                    'battery_kwh': battery_kwh,
                    'pv_kw': total_pv_capacity
                }
                # Evaluate the solution using your objective function
                # returns [-IRR, NPC]
                F[i, :] = evaluate_solution(params, self.pv_profile, self.demand_profile)
            
            # Set the output objectives array
            out["F"] = F
    
    # Create an instance of the problem
    problem = BatteryOptimizationProblem()
    print("Optimisation problem defined")
    
except Exception as e:
    print(f"Error defining optimization problem: {e}")
    raise

PETER: ADD SOLAR PV TO VARY FOR EACH ROOF
PETER: TO ADD EACH SIMULATION TO SAY TIME TAKEN
Defining optimization problem
Optimisation problem defined


In [13]:
# ─── Cell 10: Run NSGA‑II with live best‑solution logging ────────────────────
from pymoo.core.callback import Callback

class BestSolutionCallback(Callback):
    def __init__(self):
        super().__init__(frequency=1)  # once per generation

    def notify(self, algorithm):
        F = algorithm.pop.get("F")      # shape (pop_size, 2): [–IRR, NPC]
        X = algorithm.pop.get("X")[:,0] # battery sizes

        # best IRR solution
        idx_irr = np.argmin(F[:,0])
        best_irr = -F[idx_irr,0]
        batt_irr = X[idx_irr]

        # best NPC solution
        idx_npc = np.argmin(F[:,1])
        best_npc = F[idx_npc,1]
        batt_npc = X[idx_npc]

        print(
            f"Gen {algorithm.n_gen:>2d} | "
            f"Best IRR → {best_irr*100:5.2f}% @ {batt_irr:5.1f} kWh;  "
            f"Best NPC → ${best_npc:,.0f} @ {batt_npc:5.1f} kWh"
        )

# now run just once:
print("Running NSGA‑II optimisation…")

res = minimize(
    problem,
    NSGA2(pop_size=40),
    get_termination("n_gen", 50),
    seed=42,
    verbose=False,
    callback=BestSolutionCallback()
)

# extract & save Pareto front
import pandas as pd
battery_sizes = res.X.flatten()
pareto_F      = res.F
irr_vals      = -pareto_F[:,0]
npc_vals      = pareto_F[:,1]

df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'IRR':          irr_vals,
    'NPC':          npc_vals
})

(out_dir / 'pareto_solutions.csv').write_text(df.to_csv(index=False))
print(f"\n✅ Pareto front saved to {out_dir/'pareto_solutions.csv'}")


Running NSGA‑II optimisation…


TypeError: Callback.__init__() got an unexpected keyword argument 'frequency'

In [ ]:
# Cell 11: Visualise/showcase result
# PeTER fix: the output not defiend, re do but I want it in a seperate, also we want the half hourly results for 30 years to proof with import and export, 
# would be beneficial to have, in csv
print("Creating visualization of Pareto front...")

try:
    import matplotlib.pyplot as plt
    
    # Plot the Pareto front
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(df['IRR'] * 100, df['NPC'] / 1000, 
                          c=df['battery_kwh'], cmap='viridis', 
                          s=100, alpha=0.7)
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Battery Size (kWh)')
    
    # Add labels for points
    for i, row in df.iterrows():
        plt.annotate(f"{row['battery_kwh']:.1f} kWh", 
                     (row['IRR'] * 100, row['NPC'] / 1000),
                     xytext=(5, 5), textcoords='offset points')
    
    # Add labels and title
    plt.xlabel('IRR (%)')
    plt.ylabel('Net Present Cost ($ thousands)')
    plt.title('Pareto Front: IRR vs. NPC for Different Battery Sizes')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Save the figure
    plt.tight_layout()
    out_fig = out_dir / 'pareto_front.png'
    plt.savefig(out_fig, dpi=300)
    plt.show()
    
    print(f"Visualization saved to {out_fig}")
    
except Exception as e:
    print(f"Error creating visualization: {e}")
    
print("\nAnalysis complete!")

In [ ]:
# ─── Cell 12: Export all inputs & results to CSV ────────────────────────────────
import pandas as pd

# 1) Roof configuration
pd.DataFrame(roof_params) \
  .to_csv(out_dir / 'roof_config.csv', index=False)

# 2) Weather files list
pd.DataFrame({'weather_file': weather_files}) \
  .to_csv(out_dir / 'weather_files.csv', index=False)

# 3) Demand file used
pd.DataFrame({'demand_file': [str(demand_file)]}) \
  .to_csv(out_dir / 'demand_file.csv', index=False)

# 4) Simulation parameters
pd.DataFrame({
    'repeats_per_file': [REPEATS_PER_FILE],
    'start_years':        [start_years]
}) \
  .to_csv(out_dir / 'simulation_params.csv', index=False)

# 5) Pareto front (raw)
df.to_csv(out_dir / 'pareto_solutions.csv', index=False)

# 6) Pick the “best” solution (here: highest IRR) and recompute totals/financials
best_idx    = df['IRR'].idxmax()
best_batt   = df.loc[best_idx, 'battery_kwh']
print(f"Exporting full dispatch & financials for best IRR battery = {best_batt:.1f} kWh")

# 6a) Dispatch over 30 yrs for that battery
from battery_30 import simulate_battery_dispatch
from financial_30 import compute_financials

# align profiles
pv_ser   = pv_profile['simulated_kwh'] if hasattr(pv_profile, 'columns') else pv_profile
disp_df, totals = simulate_battery_dispatch(
    pv_gen      = pv_ser,
    demand      = demand_profile,
    battery_kwh = best_batt
)

# export full half‑hour series
disp_df.to_csv(out_dir / 'full_dispatch_time_series.csv')

# 6b) financial summary
fin_results = compute_financials(
    totals,
    battery_kwh = best_batt,
    pv_kw       = total_pv_capacity
)

pd.DataFrame.from_dict(fin_results, orient='index', columns=['value']) \
  .to_csv(out_dir / 'financial_summary.csv')

# 7) Dispatch aggregates & emissions
pd.DataFrame.from_dict(totals, orient='index', columns=['value']) \
  .to_csv(out_dir / 'dispatch_totals.csv')

print(f"✅ All summary CSVs written to {out_dir}")
